# Web Scraping com BS4


Estudo da Biblioteca BeautifulSoup para Web Scraping

Importando bibliotecas

In [111]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Lendo a primeira página

In [112]:
url = "https://books.toscrape.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}

page = requests.get(url,headers=headers)

In [113]:
type(page.text)

str

In [114]:
soup = BeautifulSoup(page.text, "html.parser")
type(soup)

bs4.BeautifulSoup

Procurando os nomes dos livros da página

In [115]:
book_title = soup.find_all("a")
book_title[54].get("title")

'A Light in the Attic'

In [116]:
title_soup = soup.find_all("a")

title_list = [title.get('title') for i, title in enumerate(title_soup) if i >= 54 and i % 2 == 0 ]

Procurando os preços dos livros

In [117]:
priceSoup = soup.find_all("p", class_="price_color")
price_list = [float(price.string.replace('Â£', '')) for price in priceSoup] 

Obtendo a informação se o livro está em estoque

In [118]:
inStockSoupe = soup.find_all("p", class_="instock availability")
inStocklist = []
for element in inStockSoupe:
    inStocklist.append('In stock' in element.text)

Transformando a informação em DataFrame

In [119]:
bookDict = {'Title': title_list,
          'Price': price_list,
          'InStock': inStocklist}

bookDF = pd.DataFrame(bookDict)
bookDF['Page'] = 1

In [120]:
bookDF

,Title,Price,InStock,Page
0,A Light in the Attic,51.77,True,1
1,Tipping the Velvet,53.74,True,1
2,Soumission,50.10,True,1
3,Sharp Objects,47.82,True,1
4,Sapiens: A Brief History of Humankind,54.23,True,1
5,The Requiem Red,22.65,True,1
6,The Dirty Little Secrets of Getting Your Dream...,33.34,True,1
7,The Coming Woman: A Novel Based on the Life of...,17.93,True,1
8,The Boys in the Boat: Nine Americans and Their...,22.60,True,1
9,The Black Maria,52.15,True,1


Realizando a mesma operação para as demais páginas

In [121]:
for i in range(2,50):
    link = f'https://books.toscrape.com/catalogue/page-{i}.html'
    pages = requests.get(link,headers=headers)
    
    soup = BeautifulSoup(pages.text, "html.parser")
    
    title_soup = soup.find_all("a")

    title_list = [title.get('title') for i, title in enumerate(title_soup) if i >= 54 and i % 2 == 0 ]
    title_list.pop()
    
    priceSoup = soup.find_all("p", class_="price_color")
    price_list = [float(price.string.replace('Â£', '')) for price in priceSoup]
    
    inStockSoupe = soup.find_all("p", class_="instock availability")
    inStocklist = []
    for element in inStockSoupe:
      inStocklist.append('In stock' in element.text)
    
    bookDict = {'Title': title_list,
          'Price': price_list,
          'InStock': inStocklist}

    bookDF_aux = pd.DataFrame(bookDict)
    bookDF_aux['Page'] = i
    
    bookDF = pd.concat([bookDF,bookDF_aux])

DataFrame final com todas as informações

In [123]:
bookDF

,Title,Price,InStock,Page
0,A Light in the Attic,51.77,True,1
1,Tipping the Velvet,53.74,True,1
2,Soumission,50.10,True,1
3,Sharp Objects,47.82,True,1
4,Sapiens: A Brief History of Humankind,54.23,True,1
...,...,...,...,...
15,Icing (Aces Hockey #2),40.44,True,49
16,"Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)",45.24,True,49
17,Having the Barbarian's Baby (Ice Planet Barbar...,34.96,True,49
18,"Giant Days, Vol. 1 (Giant Days #1-4)",56.76,True,49
